In [5]:
import pandas as pd
import numpy as np
import os
import random
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.regularizers import l1 , l2

import os
import numpy as np
import shutil
import random


Using TensorFlow backend.


In [0]:
#this section of code downloads the dataset from kaggle

!pip install -U -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d moltean/fruits
!ls

 99% 676M/684M [00:26<00:00, 33.7MB/s]
100% 684M/684M [00:26<00:00, 27.1MB/s]
fruits.zip  kaggle.json  sample_data


In [0]:
#Unzips the data

!unzip -q "fruits.zip"
!ls

fruits-360  fruits.zip	kaggle.json  sample_data


In [0]:
#Creats the Fiolders for the new dataset with 4 classes, apple, orange, lemon and pear
#The dataset will be divided into train validation and test set.

!rm -r NewDataSet
!mkdir NewDataSet
!mkdir NewDataSet/Train
!mkdir NewDataSet/Test
!mkdir NewDataSet/Validation
!mkdir NewDataSet/Train/Apple
!mkdir NewDataSet/Train/Lemon
!mkdir NewDataSet/Train/Orange
!mkdir NewDataSet/Train/Pear
!mkdir NewDataSet/Test/Apple
!mkdir NewDataSet/Test/Lemon
!mkdir NewDataSet/Test/Orange
!mkdir NewDataSet/Test/Pear
!mkdir NewDataSet/Validation/Apple
!mkdir NewDataSet/Validation/Lemon
!mkdir NewDataSet/Validation/Orange
!mkdir NewDataSet/Validation/Pear


!ls

rm: cannot remove 'NewDataSet': No such file or directory
fruits-360  fruits.zip	kaggle.json  NewDataSet  sample_data


In [0]:
#Copies the images from the training folder of the kaggle dataset into the folders
#of the new dataset


#Creates a list of the directories inside the training folder and sortes them 
#alphabetically
list_dir = os.listdir('fruits-360/Training')
directories=sorted(list_dir)

#Function that receives the fruit, percentage of data that we want to copy, and
#first and last folder we want to copy from, and copys a certain percentge of the images
#into the new folder
#it´s not random to make sure we get images from every view
def train_fruit(fruit, percentage, begin, end):
  for i in range(begin,end):
    allFiles = os.listdir('fruits-360/Training/'+directories[i]) 
    allFiles=sorted(allFiles)
 
    for ii in range(0, len(allFiles),int(1/percentage)):
      s='fruits-360/Training/' + directories[i]+ '/'+ allFiles[ii]
      d='NewDataSet/Train/'+fruit+'/'+str(i)+allFiles[ii]
      shutil.copyfile(s,d)
      

train_fruit("Apple", 1, 0,12)
train_fruit("Lemon", 1, 49,50 )
train_fruit("Orange",1, 60,61 )
train_fruit("Pear", 1, 66,71)


NameError: ignored

In [0]:
#Moves a certaing percentage of the training set images the validation folder

  
#function that receives the name of the fruit and the percentage of the taining 
#set to used for validation and moves the images acording to those parameters
  def validation_fruit(fruit, percentage):
    allFiles = os.listdir('NewDataSet/Train/'+fruit) 
    allFiles=sorted(allFiles)
    a=int(len(allFiles)*percentage)
    r=random.sample(range(1, len(allFiles)), a)

    for i in range(0,len(r)):
        s='NewDataSet/Train/'+fruit+'/'+allFiles[r[i]]
        d='NewDataSet/Validation/'+fruit+'/'+str(i)+allFiles[r[i]]
        if os.path.exists(s):
          shutil.move(s,d)
      
validation_fruit("Apple",0.25)
validation_fruit("Lemon",0.25)
validation_fruit("Orange",0.25)
validation_fruit("Pear",0.25)
      

In [0]:
#Copies the images from the test folder of the kaggle dataset into the folders
#of the new dataset


#Creates a list of the directories inside the test folder and sortes them 
#alphabetically
list_dir = os.listdir('fruits-360/Test')
directories=sorted(list_dir)

#Function that receives the fruit, percentage of data that we want to copy, and
#first and last folder we want to copy from, and copys a certain percentge of the images
#into the new folder
#it´s not random to make sure we get images from every view
def train_fruit(fruit, percentage, begin, end):
  for i in range(begin,end):
    allFiles = os.listdir('fruits-360/Test/'+directories[i]) 
    allFiles=sorted(allFiles)
 
    for ii in range(0, len(allFiles),int(1/percentage)):
      s='fruits-360/Test/' + directories[i]+ '/'+ allFiles[ii]
      d='NewDataSet/Test/'+fruit+'/'+str(i)+allFiles[ii]
      shutil.copyfile(s,d)
      

train_fruit("Apple", 1, 0,12)
train_fruit("Lemon", 1, 49,50 )
train_fruit("Orange",1, 60,61 )
train_fruit("Pear", 1, 66,71)

In [6]:
#formating the training, validation and test sets
#we use ImageDataGenerator to fetch data and feed it to our network

train_data_dir="NewDataSet/Train"
validation_data_dir="NewDataSet/Validation"
test_data_dir="NewDataSet/Test"

batch_size=32


train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) 
train_generator=train_datagen.flow_from_directory(train_data_dir,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=True)
validation_generator=train_datagen.flow_from_directory(validation_data_dir,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=True)
test_generator=train_datagen.flow_from_directory(test_data_dir,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=True)



Found 6863 images belonging to 4 classes.
Found 2286 images belonging to 4 classes.
Found 3039 images belonging to 4 classes.


In [0]:
#keras.applications.mobilenet.MobileNet(input_shape=None, alpha=1.0, depth_multiplier=1, dropout=1e-3, include_top=True, weights='imagenet', input_tensor=None, pooling=None, classes=1000)
#keras.layers.Dense(units, activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)

#Build the model trains the network and calculates the loss and accuracy for the training set and validation set

def func(kini, bini, kreg, breg, alphas, ep):

  base_model=keras.applications.mobilenet.MobileNet(weights='imagenet',include_top=False) 
  #imports the mobilenet model and discards the last neuron layer and uses the already trained weights

  x=base_model.output
  x=GlobalAveragePooling2D()(x)
  x=Dense(1024,activation='relu',kernel_initializer=kini, bias_initializer=bini, kernel_regularizer=kreg, bias_regularizer=breg )(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
  x=Dense(1024,activation='relu',kernel_initializer=kini, bias_initializer=bini, kernel_regularizer=kreg, bias_regularizer=breg)(x) #dense layer 2
  x=Dense(512,activation='relu',kernel_initializer=kini, bias_initializer=bini, kernel_regularizer=kreg, bias_regularizer=breg)(x) #dense layer 3
  preds=Dense(4,activation='softmax')(x) #final layer with softmax activation and 4 classes
  
  model=Model(inputs=base_model.input,outputs=preds)
  model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
  step_size_train=train_generator.n//train_generator.batch_size
  step_size_validation=validation_generator.n//train_generator.batch_size
  model.fit_generator(generator=train_generator,
                      validation_data=validation_generator,
                      validation_steps=step_size_validation,
                      steps_per_epoch=step_size_train,
                      epochs=ep,
                      callbacks=[EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto', baseline=None, restore_best_weights=True)]
                     )
  

In [0]:
def func1(kini, bini, kreg, breg, alphas, ep):

  base_model=keras.applications.mobilenet.MobileNet(weights='imagenet',include_top=False) 
  #imports the mobilenet model and discards the last neuron layer and uses the already trained weights

  x=base_model.output
  x=GlobalAveragePooling2D()(x)
  x=Dense(1024,activation='relu',kernel_initializer=kini, bias_initializer=bini, kernel_regularizer=kreg, bias_regularizer=breg )(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
  preds=Dense(4,activation='softmax')(x) #final layer with softmax activation and 4 classes
  
  model=Model(inputs=base_model.input,outputs=preds)
  model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
  step_size_train=train_generator.n//train_generator.batch_size
  step_size_validation=validation_generator.n//train_generator.batch_size
  model.fit_generator(generator=train_generator,
                      validation_data=validation_generator,
                      validation_steps=step_size_validation,
                      steps_per_epoch=step_size_train,
                      epochs=ep,
                      callbacks=[EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto', baseline=None, restore_best_weights=True)]
                     )
  

In [0]:
def func6(kini, bini, kreg, breg, alphas, ep):

  base_model=keras.applications.mobilenet.MobileNet(weights='imagenet',include_top=False) 
  #imports the mobilenet model and discards the last neuron layer and uses the already trained weights

  x=base_model.output
  x=GlobalAveragePooling2D()(x)
  x=Dense(1024,activation='relu',kernel_initializer=kini, bias_initializer=bini, kernel_regularizer=kreg, bias_regularizer=breg )(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
  x=Dense(1024,activation='relu',kernel_initializer=kini, bias_initializer=bini, kernel_regularizer=kreg, bias_regularizer=breg )(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
  x=Dense(1024,activation='relu',kernel_initializer=kini, bias_initializer=bini, kernel_regularizer=kreg, bias_regularizer=breg )(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
  x=Dense(1024,activation='relu',kernel_initializer=kini, bias_initializer=bini, kernel_regularizer=kreg, bias_regularizer=breg )(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
  x=Dense(1024,activation='relu',kernel_initializer=kini, bias_initializer=bini, kernel_regularizer=kreg, bias_regularizer=breg )(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
  x=Dense(512,activation='relu',kernel_initializer=kini, bias_initializer=bini, kernel_regularizer=kreg, bias_regularizer=breg )(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
  preds=Dense(4,activation='softmax')(x) #final layer with softmax activation and 4 classes
  
  model=Model(inputs=base_model.input,outputs=preds)
  model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
  step_size_train=train_generator.n//train_generator.batch_size
  step_size_validation=validation_generator.n//train_generator.batch_size
  model.fit_generator(generator=train_generator,
                      validation_data=validation_generator,
                      validation_steps=step_size_validation,
                      steps_per_epoch=step_size_train,
                      epochs=ep,
                      callbacks=[EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto', baseline=None, restore_best_weights=True)]
                     )
  

In [0]:
func('glorot_uniform', 'zeros', None , None,1,20)

Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


Instructions for updating:
Use tf.cast instead.
Epoch 1/20
214/214 [==============================] - 61s 286ms/step - loss: 0.1059 - acc: 0.9701 - val_loss: 1.5561 - val_acc: 0.8706
Epoch 2/20
214/214 [==============================] - 53s 246ms/step - loss: 0.0883 - acc: 0.9804 - val_loss: 1.0983 - val_acc: 0.8421
Epoch 3/20
214/214 [==============================] - 52s 244ms/step - loss: 0.0458 - acc: 0.9877 - val_loss: 0.3292 - val_acc: 0.9441
Epoch 4/20
214/214 [==============================] - 52s 244ms/step - loss: 0.0073 - acc: 0.9987 - val_loss: 9.9648e-07 - val_acc: 1.0000
Epoch 5/20
214/214 [==============================] - 52s 243ms/step - loss: 0.0092 - acc: 0.9991 - val_loss: 1.8208e-05 - val_acc: 1.0000
Epoch 6/20
214/214 [==============================] - 52s 243ms/step - loss: 0.0584 - acc: 0.9860 - val_loss: 0.0250 - val_acc: 0.9911
Epoch 7/20
214/214 [==============================] - 52s 244ms/step - loss: 0.0477 - acc: 0.9917 - val_loss: 0.0146 - val_acc: 0.9991

In [0]:
func6('glorot_uniform', 'zeros', None , None,1,20)

Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


17227776/17225924 [==============================] - 0s 0us/step
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
214/214 [==============================] - 59s 277ms/step - loss: 0.2326 - acc: 0.9011 - val_loss: 1.0260 - val_acc: 0.9010
Epoch 2/20
214/214 [==============================] - 51s 237ms/step - loss: 0.1155 - acc: 0.9607 - val_loss: 0.0285 - val_acc: 0.9942
Epoch 3/20
214/214 [==============================] - 51s 237ms/step - loss: 0.4650 - acc: 0.9039 - val_loss: 1.4397 - val_acc: 0.6513
Epoch 4/20
214/214 [==============================] - 51s 239ms/step - loss: 0.1520 - acc: 0.9388 - val_loss: 0.1572 - val_acc: 0.9450
Epoch 5/20
214/214 [==============================] - 51s 240ms/step - loss: 0.3331 - acc: 0.9530 - val_loss: 5.9495 - val_acc: 0.6309


In [15]:
func1('glorot_uniform', 'zeros', None , None,1,20)

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


Epoch 1/20
214/214 [==============================] - 62s 289ms/step - loss: 0.0867 - acc: 0.9779 - val_loss: 3.3297 - val_acc: 0.5775
Epoch 2/20
214/214 [==============================] - 52s 244ms/step - loss: 0.0260 - acc: 0.9911 - val_loss: 0.0583 - val_acc: 0.9902
Epoch 3/20
214/214 [==============================] - 52s 242ms/step - loss: 0.0067 - acc: 0.9984 - val_loss: 4.8710e-04 - val_acc: 1.0000
Epoch 4/20
214/214 [==============================] - 52s 243ms/step - loss: 0.0288 - acc: 0.9918 - val_loss: 0.0684 - val_acc: 0.9791
Epoch 5/20
214/214 [==============================] - 52s 242ms/step - loss: 0.0061 - acc: 0.9982 - val_loss: 7.3497e-05 - val_acc: 1.0000
Epoch 6/20
214/214 [==============================] - 52s 243ms/step - loss: 7.8273e-05 - acc: 1.0000 - val_loss: 1.9603e-06 - val_acc: 1.0000
Epoch 7/20
214/214 [==============================] - 52s 243ms/step - loss: 1.9827e-05 - acc: 1.0000 - val_loss: 6.6614e-07 - val_acc: 1.0000
Epoch 8/20
214/214 [===========

In [16]:
func1('zero', 'zeros', None , None,1,20)

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


Epoch 1/20
214/214 [==============================] - 64s 297ms/step - loss: 1.3121 - acc: 0.6279 - val_loss: 1.2451 - val_acc: 0.6278
Epoch 2/20
214/214 [==============================] - 51s 237ms/step - loss: 1.1928 - acc: 0.6269 - val_loss: 1.1447 - val_acc: 0.6290
Epoch 3/20
214/214 [==============================] - 51s 237ms/step - loss: 1.1093 - acc: 0.6279 - val_loss: 1.0774 - val_acc: 0.6278
Epoch 4/20
214/214 [==============================] - 51s 237ms/step - loss: 1.0541 - acc: 0.6271 - val_loss: 1.0339 - val_acc: 0.6256
Epoch 5/20
214/214 [==============================] - 51s 237ms/step - loss: 1.0168 - acc: 0.6280 - val_loss: 0.9989 - val_acc: 0.6304
Epoch 6/20
214/214 [==============================] - 50s 236ms/step - loss: 0.9957 - acc: 0.6252 - val_loss: 0.9829 - val_acc: 0.6287
Epoch 7/20
214/214 [==============================] - 51s 236ms/step - loss: 0.9774 - acc: 0.6306 - val_loss: 0.9838 - val_acc: 0.6216
Epoch 8/20
214/214 [==============================] - 5

In [17]:
func1('ones', 'zeros', None , None,1,20)

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


Epoch 1/20
214/214 [==============================] - 65s 305ms/step - loss: 15.2731 - acc: 0.0524 - val_loss: 15.2886 - val_acc: 0.0515
Epoch 2/20
214/214 [==============================] - 50s 235ms/step - loss: 15.2778 - acc: 0.0521 - val_loss: 15.3029 - val_acc: 0.0506
Epoch 3/20
214/214 [==============================] - 50s 236ms/step - loss: 15.2749 - acc: 0.0523 - val_loss: 15.2671 - val_acc: 0.0528
Epoch 4/20
214/214 [==============================] - 51s 238ms/step - loss: 15.2708 - acc: 0.0526 - val_loss: 15.2600 - val_acc: 0.0532
Epoch 5/20
214/214 [==============================] - 51s 236ms/step - loss: 15.2822 - acc: 0.0519 - val_loss: 15.2886 - val_acc: 0.0515
Epoch 6/20
214/214 [==============================] - 51s 238ms/step - loss: 15.2490 - acc: 0.0539 - val_loss: 15.2886 - val_acc: 0.0515
Epoch 7/20
214/214 [==============================] - 51s 239ms/step - loss: 15.2755 - acc: 0.0523 - val_loss: 15.2600 - val_acc: 0.0532
Epoch 8/20
214/214 [=====================

In [18]:
func1('RandomNormal', 'zeros', None , None,1,20)

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


Epoch 1/20
214/214 [==============================] - 70s 325ms/step - loss: 0.1027 - acc: 0.9749 - val_loss: 0.4639 - val_acc: 0.8684
Epoch 2/20
214/214 [==============================] - 52s 245ms/step - loss: 4.9892e-04 - acc: 0.9999 - val_loss: 0.4060 - val_acc: 0.9139
Epoch 3/20
214/214 [==============================] - 53s 246ms/step - loss: 0.0472 - acc: 0.9873 - val_loss: 0.1024 - val_acc: 0.9672
Epoch 4/20
214/214 [==============================] - 53s 246ms/step - loss: 0.0134 - acc: 0.9968 - val_loss: 0.2923 - val_acc: 0.9170
Epoch 5/20
214/214 [==============================] - 53s 246ms/step - loss: 0.0311 - acc: 0.9914 - val_loss: 7.3140e-06 - val_acc: 1.0000
Epoch 6/20
214/214 [==============================] - 53s 247ms/step - loss: 3.6680e-04 - acc: 0.9999 - val_loss: 1.4887e-06 - val_acc: 1.0000
Epoch 7/20
214/214 [==============================] - 53s 245ms/step - loss: 0.0069 - acc: 0.9985 - val_loss: 6.3461e-07 - val_acc: 1.0000
Epoch 8/20
214/214 [===============

In [19]:
func1('RandomUniform', 'zeros', None , None,1,20)

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


Epoch 1/20
214/214 [==============================] - 73s 339ms/step - loss: 0.1143 - acc: 0.9732 - val_loss: 0.0320 - val_acc: 0.9867
Epoch 2/20
214/214 [==============================] - 52s 245ms/step - loss: 0.0097 - acc: 0.9966 - val_loss: 8.5952e-06 - val_acc: 1.0000
Epoch 3/20
214/214 [==============================] - 52s 244ms/step - loss: 0.0284 - acc: 0.9917 - val_loss: 0.3805 - val_acc: 0.9219
Epoch 4/20
214/214 [==============================] - 52s 245ms/step - loss: 0.0146 - acc: 0.9959 - val_loss: 0.0018 - val_acc: 0.9987
Epoch 5/20
214/214 [==============================] - 52s 245ms/step - loss: 0.0104 - acc: 0.9972 - val_loss: 0.0099 - val_acc: 0.9973


In [32]:
func1('glorot_uniform', 'zeros', l2(0.1), None,1,20)

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


Epoch 1/20
214/214 [==============================] - 86s 403ms/step - loss: 8.7042 - acc: 0.9708 - val_loss: 2.8989 - val_acc: 0.5878
Epoch 2/20
214/214 [==============================] - 53s 250ms/step - loss: 0.0798 - acc: 0.9934 - val_loss: 0.6227 - val_acc: 0.9224
Epoch 3/20
214/214 [==============================] - 53s 250ms/step - loss: 0.0433 - acc: 0.9988 - val_loss: 0.0161 - val_acc: 1.0000
Epoch 4/20
214/214 [==============================] - 53s 248ms/step - loss: 0.0152 - acc: 0.9999 - val_loss: 0.0103 - val_acc: 1.0000
Epoch 5/20
214/214 [==============================] - 53s 250ms/step - loss: 0.0287 - acc: 0.9988 - val_loss: 0.9658 - val_acc: 0.9348
Epoch 6/20
214/214 [==============================] - 53s 249ms/step - loss: 0.4786 - acc: 0.9666 - val_loss: 0.0621 - val_acc: 0.9867
Epoch 7/20
214/214 [==============================] - 53s 249ms/step - loss: 0.0852 - acc: 0.9946 - val_loss: 1.6483 - val_acc: 0.6131


In [33]:
func1('glorot_uniform', 'zeros', l1(0.1), None,1,20)

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


Epoch 1/20
214/214 [==============================] - 90s 419ms/step - loss: 273.7446 - acc: 0.9329 - val_loss: 14.9462 - val_acc: 0.7356
Epoch 2/20
214/214 [==============================] - 54s 253ms/step - loss: 14.3637 - acc: 0.9159 - val_loss: 15.0107 - val_acc: 0.9468
Epoch 3/20
214/214 [==============================] - 54s 251ms/step - loss: 14.5122 - acc: 0.9206 - val_loss: 14.4729 - val_acc: 0.7360
Epoch 4/20
214/214 [==============================] - 54s 252ms/step - loss: 14.6738 - acc: 0.9011 - val_loss: 14.7309 - val_acc: 0.8602
Epoch 5/20
214/214 [==============================] - 54s 251ms/step - loss: 13.7740 - acc: 0.9274 - val_loss: 15.6973 - val_acc: 0.8980
Epoch 6/20
214/214 [==============================] - 54s 251ms/step - loss: 14.5156 - acc: 0.9146 - val_loss: 13.1549 - val_acc: 0.9437
Epoch 7/20
214/214 [==============================] - 54s 252ms/step - loss: 13.7604 - acc: 0.9270 - val_loss: 13.5606 - val_acc: 0.9299
Epoch 8/20
214/214 [====================

In [34]:
func1('glorot_uniform', 'zeros', None, None,0.9,20)

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


Epoch 1/20
214/214 [==============================] - 93s 434ms/step - loss: 0.0928 - acc: 0.9782 - val_loss: 0.0167 - val_acc: 0.9965
Epoch 2/20
214/214 [==============================] - 55s 256ms/step - loss: 0.0472 - acc: 0.9867 - val_loss: 1.9611 - val_acc: 0.7418
Epoch 3/20
214/214 [==============================] - 54s 251ms/step - loss: 0.0203 - acc: 0.9940 - val_loss: 0.1103 - val_acc: 0.9672
Epoch 4/20
214/214 [==============================] - 54s 253ms/step - loss: 3.0425e-04 - acc: 1.0000 - val_loss: 9.8497e-04 - val_acc: 0.9996
Epoch 5/20
214/214 [==============================] - 54s 252ms/step - loss: 0.0119 - acc: 0.9963 - val_loss: 9.5081e-04 - val_acc: 1.0000
Epoch 6/20
214/214 [==============================] - 54s 252ms/step - loss: 0.0083 - acc: 0.9981 - val_loss: 0.0011 - val_acc: 0.9996
Epoch 7/20
214/214 [==============================] - 54s 252ms/step - loss: 9.7928e-05 - acc: 1.0000 - val_loss: 7.2368e-07 - val_acc: 1.0000
Epoch 8/20
214/214 [===============

In [35]:
func1('glorot_uniform', 'zeros', None, None,0.75,20)

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


Epoch 1/20
214/214 [==============================] - 95s 445ms/step - loss: 0.0949 - acc: 0.9782 - val_loss: 7.2342e-05 - val_acc: 1.0000
Epoch 2/20
214/214 [==============================] - 55s 258ms/step - loss: 0.0515 - acc: 0.9895 - val_loss: 1.7251e-05 - val_acc: 1.0000
Epoch 3/20
214/214 [==============================] - 55s 257ms/step - loss: 0.0013 - acc: 0.9996 - val_loss: 4.2113e-05 - val_acc: 1.0000
Epoch 4/20
214/214 [==============================] - 55s 257ms/step - loss: 2.7356e-04 - acc: 0.9999 - val_loss: 3.6436e-04 - val_acc: 1.0000
Epoch 5/20
214/214 [==============================] - 55s 257ms/step - loss: 5.7043e-05 - acc: 1.0000 - val_loss: 1.0295e-06 - val_acc: 1.0000
Epoch 6/20
214/214 [==============================] - 55s 257ms/step - loss: 1.4250e-05 - acc: 1.0000 - val_loss: 6.3283e-07 - val_acc: 1.0000
Epoch 7/20
214/214 [==============================] - 55s 257ms/step - loss: 7.9393e-06 - acc: 1.0000 - val_loss: 4.7116e-07 - val_acc: 1.0000
Epoch 8/20


In [36]:
func1('glorot_uniform', 'zeros', None, None,1.1,20)

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


Epoch 1/20
214/214 [==============================] - 100s 465ms/step - loss: 0.0807 - acc: 0.9772 - val_loss: 0.0918 - val_acc: 0.9752
Epoch 2/20
214/214 [==============================] - 57s 264ms/step - loss: 0.0229 - acc: 0.9925 - val_loss: 0.0194 - val_acc: 0.9956
Epoch 3/20
214/214 [==============================] - 56s 262ms/step - loss: 0.0243 - acc: 0.9940 - val_loss: 0.0641 - val_acc: 0.9814
Epoch 4/20
214/214 [==============================] - 56s 263ms/step - loss: 0.0218 - acc: 0.9953 - val_loss: 0.1769 - val_acc: 0.9658
Epoch 5/20
214/214 [==============================] - 56s 262ms/step - loss: 0.0248 - acc: 0.9913 - val_loss: 0.2647 - val_acc: 0.9215


In [21]:
kini='RandomNormal'
bini='zeros'
kreg=None
breg=None
alpha=0.75
ep=20

base_model=keras.applications.mobilenet.MobileNet(weights='imagenet',include_top=False) 
  #imports the mobilenet model and discards the last neuron layer and uses the already trained weights

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu',kernel_initializer=kini, bias_initializer=bini, kernel_regularizer=kreg, bias_regularizer=breg )(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
preds=Dense(4,activation='softmax')(x) #final layer with softmax activation and 4 classes
  
model=Model(inputs=base_model.input,outputs=preds)
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
step_size_train=train_generator.n//train_generator.batch_size
step_size_validation=validation_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                      validation_data=validation_generator,
                      validation_steps=step_size_validation,
                      steps_per_epoch=step_size_train,
                      epochs=ep,
                      callbacks=[EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto', baseline=None, restore_best_weights=True)]
                     )


/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


Epoch 1/20
214/214 [==============================] - 69s 324ms/step - loss: 0.0820 - acc: 0.9793 - val_loss: 0.2620 - val_acc: 0.9468
Epoch 2/20
214/214 [==============================] - 52s 244ms/step - loss: 0.0658 - acc: 0.9852 - val_loss: 9.7779e-04 - val_acc: 0.9996
Epoch 3/20
214/214 [==============================] - 52s 244ms/step - loss: 0.0015 - acc: 0.9996 - val_loss: 1.2325e-06 - val_acc: 1.0000
Epoch 4/20
214/214 [==============================] - 52s 245ms/step - loss: 3.7721e-05 - acc: 1.0000 - val_loss: 3.6633e-07 - val_acc: 1.0000
Epoch 5/20
214/214 [==============================] - 52s 245ms/step - loss: 0.0845 - acc: 0.9798 - val_loss: 0.0886 - val_acc: 0.9752
Epoch 6/20
214/214 [==============================] - 52s 244ms/step - loss: 3.6070e-04 - acc: 1.0000 - val_loss: 6.3151e-06 - val_acc: 1.0000
Epoch 7/20
214/214 [==============================] - 52s 244ms/step - loss: 0.0114 - acc: 0.9982 - val_loss: 0.3850 - val_acc: 0.8829


In [22]:
print(model.metrics_names)
#evaluate_generator(generator, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)
print(model.evaluate_generator(generator=test_generator,steps=test_generator.n//test_generator.batch_size))

['loss', 'acc']
[0.0032685854725554736, 0.9980053191489362]


In [17]:
kini='glorot_uniform'
bini='zeros'
kreg=None
breg=None
alpha=1
ep=20

base_model=keras.applications.mobilenet.MobileNet(weights='imagenet',include_top=False) 
  #imports the mobilenet model and discards the last neuron layer and uses the already trained weights

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu',kernel_initializer=kini, bias_initializer=bini, kernel_regularizer=kreg, bias_regularizer=breg )(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
preds=Dense(4,activation='softmax')(x) #final layer with softmax activation and 4 classes
  
model=Model(inputs=base_model.input,outputs=preds)
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
step_size_train=train_generator.n//train_generator.batch_size
step_size_validation=validation_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                      validation_data=validation_generator,
                      validation_steps=step_size_validation,
                      steps_per_epoch=step_size_train,
                      epochs=ep,
                      callbacks=[EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto', baseline=None, restore_best_weights=True)]
                     )


/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


Epoch 1/20
214/214 [==============================] - 63s 292ms/step - loss: 0.4676 - acc: 0.9527 - val_loss: 2.3899 - val_acc: 0.7955
Epoch 2/20
214/214 [==============================] - 52s 241ms/step - loss: 0.0271 - acc: 0.9934 - val_loss: 0.0039 - val_acc: 0.9987
Epoch 3/20
214/214 [==============================] - 52s 241ms/step - loss: 0.0192 - acc: 0.9953 - val_loss: 2.2186 - val_acc: 0.5741
Epoch 4/20
214/214 [==============================] - 52s 242ms/step - loss: 0.0168 - acc: 0.9947 - val_loss: 0.0207 - val_acc: 0.9916
Epoch 5/20
214/214 [==============================] - 52s 241ms/step - loss: 0.0236 - acc: 0.9940 - val_loss: 6.6948e-05 - val_acc: 1.0000
Epoch 6/20
214/214 [==============================] - 52s 241ms/step - loss: 0.0099 - acc: 0.9969 - val_loss: 0.0023 - val_acc: 0.9996
Epoch 7/20
214/214 [==============================] - 52s 241ms/step - loss: 0.0011 - acc: 0.9994 - val_loss: 2.9094e-06 - val_acc: 1.0000
Epoch 8/20
214/214 [===========================

In [18]:
print(model.metrics_names)
#evaluate_generator(generator, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)
print(model.evaluate_generator(generator=test_generator,steps=test_generator.n//test_generator.batch_size))

['loss', 'acc']
[0.0006312949559770384, 1.0]


In [24]:
kini='glorot_uniform'
bini='zeros'
kreg=None
breg=None
alpha=1
ep=20

base_model=keras.applications.mobilenet.MobileNet(weights='imagenet',include_top=False) 
  #imports the mobilenet model and discards the last neuron layer and uses the already trained weights

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu',kernel_initializer=kini, bias_initializer=bini, kernel_regularizer=kreg, bias_regularizer=breg )(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
preds=Dense(4,activation='softmax')(x) #final layer with softmax activation and 4 classes
  
model=Model(inputs=base_model.input,outputs=preds)
model.compile(optimizer=Adam(lr=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])
step_size_train=train_generator.n//train_generator.batch_size
step_size_validation=validation_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                      validation_data=validation_generator,
                      validation_steps=step_size_validation,
                      steps_per_epoch=step_size_train,
                      epochs=ep,
                      callbacks=[EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto', baseline=None, restore_best_weights=True)]
                     )

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


Epoch 1/20
214/214 [==============================] - 74s 344ms/step - loss: 0.0514 - acc: 0.9829 - val_loss: 2.0854e-04 - val_acc: 1.0000
Epoch 2/20
214/214 [==============================] - 53s 249ms/step - loss: 7.2327e-04 - acc: 1.0000 - val_loss: 1.0283e-04 - val_acc: 1.0000
Epoch 3/20
214/214 [==============================] - 53s 249ms/step - loss: 3.3654e-04 - acc: 1.0000 - val_loss: 3.0211e-05 - val_acc: 1.0000
Epoch 4/20
214/214 [==============================] - 53s 246ms/step - loss: 1.3843e-04 - acc: 1.0000 - val_loss: 1.9695e-05 - val_acc: 1.0000
Epoch 5/20
214/214 [==============================] - 53s 246ms/step - loss: 1.9018e-04 - acc: 1.0000 - val_loss: 1.4494e-05 - val_acc: 1.0000
Epoch 6/20
214/214 [==============================] - 53s 246ms/step - loss: 1.2768e-04 - acc: 1.0000 - val_loss: 1.0665e-05 - val_acc: 1.0000
Epoch 7/20
214/214 [==============================] - 53s 246ms/step - loss: 5.7005e-05 - acc: 1.0000 - val_loss: 5.0027e-06 - val_acc: 1.0000
Epo

In [25]:
print(model.metrics_names)
#evaluate_generator(generator, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)
print(model.evaluate_generator(generator=test_generator,steps=test_generator.n//test_generator.batch_size))

['loss', 'acc']
[0.003790959681868373, 0.9983377659574468]
